# Large Test Set #
## Goal ##
The goal of this test set is to show that what was found in the smaller data set is generalizable to a larger number of proteins with a wide range of sizes and available homologs, orthologs, and paralogs. This set of proteins was used previously (Marcos et al., 2011) to evaluate covariance predictions. By using this dataset it should be possible to show the generalizability of the method as well as make a better comparison to previous methods.
## Warning ##
Before attempting to use this notebook make sure that your .env file has been properly setup to reflect the correct locations of command line tools and the location of files and directories needed for execution.
### Initial Import###
This first cell performs the necessary imports required to begin this notebook.

In [1]:
from dotenv import find_dotenv, load_dotenv
try:
    dotenv_path = find_dotenv(raise_error_if_not_found=True)
except IOError:
    dotenv_path = find_dotenv(raise_error_if_not_found=True, usecwd=True)
load_dotenv(dotenv_path)
import os
import sys
sys.path.append(os.path.join(os.environ.get('PROJECT_PATH'), 'src', 'SupportingClasses'))
input_dir = os.environ.get('INPUT_PATH')

## Data Set Construction ##
The first task required to test the data set is to download the required data and construct any necessary input files for all down stream analyses.
In this case that means:
* Downloading PDB files for the proteins in our small test set.
* Extracting a query sequence from each PDB file.
* Searching for paralogs, homologs, and orthologs in a custom BLAST database built by filtering the Uniref90 database.
* Filtering the hits from the BLAST search to meet minimum and maximum length requirements, as well as minimum and maximum identity requirements.
* Building alignments using CULSTALW in both the fasta and msf formats since some of the tools which will be used for comparison need different formats.
* Filtering the alignment for maximum identity similarity between seqeunces.
* Re-aligning the filtered sequences using CLUSTALW.
This is all handeled by the DataSetGenerator class found in the src/SupportingClasses folder

In [2]:
protein_list_dir = os.path.join(input_dir, 'ProteinLists')
if not os.path.isdir(protein_list_dir):
    os.makedirs(protein_list_dir)
large_list_fn = os.path.join(protein_list_dir, 'LargeDataSet.txt')
proteins_of_interest = ['153lA', '154lA', '1a04A', '1a0bA', '1a0pA', '1ae9A', '1al3A', '1atgA', '1b7eA', '1b9mA', '1b9nA',
                        '1biaA', '1bibA', '1bl0A', '1booA', '1bslA', '1byiA', '1byqA', '1c02A', '1c52A', '1c5kA', '1c75A',
                        '1cb7A', '1ccwA', '1cp2A', '1crxA', '1crzA', '1ctjA', '1d4aA', '1d5yA', '1dadA', '1daeA', '1dagA',
                        '1dahA', '1daiA', '1dakA', '1dd9A', '1ddeA', '1di6A', '1di7A', '1dljA', '1dtsA', '1durA', '1e2xA',
                        '1e3uA', '1e4dA', '1e4fA', '1e4gA', '1e8cA', '1eclA', '1efaA', '1efdA', '1eg2A', '1ek9A', '1eszA',
                        '1etkA', '1etoA', '1etvA', '1etwA', '1etxA', '1etyA', '1ezwA', '1f07A', '1f1uA', '1f44A', '1f48A',
                        '1f5vA', '1f9iA', '1fcaA', '1fdnA', '1fepA', '1fiaA', '1fipA', '1fp6A', '1fr3A', '1fseA', '1fxoA',
                        '1g1lA', '1g1mA', '1g20A', '1g28A', '1g5pA', '1g60A', '1g6oA', '1g72A', '1g8kA', '1gbsA', '1gdtA',
                        '1gg4A', '1gqyA', '1gu9A', '1gugA', '1gunA', '1gusA', '1gutA', '1h3lA', '1h4iA', '1h7lA', '1h7qA',
                        '1h8zA', '1h98A', '1h9gA', '1h9jA', '1h9kA', '1h9mA', '1h9sA', '1hfeA', '1hm9A', '1hw1A', '1hxdA',
                        '1i0rA', '1i1gA', '1i52A', '1i58A', '1i5nA', '1i74A', '1i8oA', '1i9cA', '1icrA', '1id0A', '1id1A',
                        '1ihcA', '1ihrA', '1ihuA', '1ii0A', '1ii9A', '1iniA', '1injA', '1ir6A', '1iujA', '1ixcA', '1ixgA',
                        '1ixhA', '1iz1A', '1j5yA', '1j6uA', '1jbgA', '1jbwA', '1je8A', '1jetA', '1jeuA', '1jevA', '1jftA',
                        '1jh9A', '1jiwA', '1jljA', '1jnuA', '1jpuA', '1jq5A', '1jykA', '1k20A', '1k2vA', '1k38A', '1k4fA',
                        '1k54A', '1k56A', '1kapA', '1kb0A', '1kbuA', '1kgsA', '1kmoA', '1kmpA', '1kq3A', '1ku3A', '1ku7A',
                        '1kv9A', '1kw3A', '1kw6A', '1l3lA', '1lj9A', '1lq9A', '1lqkA', '1lqpA', '1lr0A', '1ls9A', '1lspA',
                        '1lssA', '1lucA', '1lvwA', '1m65A', '1m68A', '1m6kA', '1m70A', '1m7jA', '1ma7A', '1mb3A', '1mdoA',
                        '1mkmA', '1mkzA', '1mm8A', '1mnzA', '1moqA', '1muhA', '1murA', '1musA', '1muwA', '1mv8A', '1mw8A',
                        '1mw9A', '1n2zA', '1n9lA', '1n9nA', '1nfpA', '1nkiA', '1nlyA', '1nnfA', '1noxA', '1nqeA', '1nw5A',
                        '1nw6A', '1nw7A', '1nw8A', '1nwzA', '1ny5A', '1ny6A', '1o1hA', '1o2dA', '1o61A', '1o69A', '1o7lA',
                        '1oadA', '1oapA', '1oddA', '1odvA', '1oj7A', '1oltA', '1opcA', '1opxA', '1or7A', '1ot6A', '1ot9A',
                        '1otaA', '1otbA', '1oxkA', '1p2fA', '1p31A', '1p3dA', '1p7dA', '1p9rA', '1p9wA', '1pb0A', '1pb7A',
                        '1pb8A', '1pjrA', '1pnzA', '1po0A', '1pt7A', '1pvpA', '1q05A', '1q06A', '1q07A', '1q08A', '1q09A',
                        '1q0aA', '1q35A', '1q7eA', '1qg8A', '1qgqA', '1qgsA', '1qhgA', '1qhhA', '1qksA', '1qpzA', '1qsaA',
                        '1qteA', '1qtwA', '1qu7A', '1qwyA', '1qxxA', '1r1mA', '1r1tA', '1r1uA', '1r23A', '1r62A', '1r8dA',
                        '1r8eA', '1r9xA', '1r9yA', '1r9zA', '1ra0A', '1ra5A', '1rakA', '1reqA', '1rhcA', '1rioA', '1rk6A',
                        '1rp3A', '1rrmA', '1rttA', '1rzuA', '1rzvA', '1s5mA', '1s5nA', '1s8nA', '1sfxA', '1sg0A', '1si0A',
                        '1sigA', '1slyA', '1sqeA', '1sqsA', '1sumA', '1suuA', '1t3tA', '1t5bA', '1t72A', '1ta9A', '1td5A',
                        '1tf1A', '1tqgA', '1tqqA', '1tv8A', '1tvlA', '1tzbA', '1tzcA', '1u07A', '1u2wA', '1u8bA', '1u8tA',
                        '1uaaA', '1uc8A', '1uc9A', '1us4A', '1us5A', '1uscA', '1usfA', '1uuxA', '1uuyA', '1uylA', '1v4yA',
                        '1v51A', '1v8pA', '1v96A', '1vctA', '1ve2A', '1vf7A', '1vgtA', '1vgwA', '1vhdA', '1vhvA', '1vimA',
                        '1vj7A', '1vkeA', '1vljA', '1vz0A', '1w55A', '1w6sA', '1w77A', '1w78A', '1w8iA', '1wetA', '1wmiA',
                        '1woqA', '1wp1A', '1wpmA', '1wpnA', '1wppA', '1ws6A', '1x74A', '1x9hA', '1x9iA', '1xa3A', '1xc3A',
                        '1xd7A', '1xi2A', '1xjaA', '1xk6A', '1xk7A', '1xkwA', '1xkzA', '1xmaA', '1xo0A', '1xocA', '1xw3A',
                        '1y0hA', '1y1zA', '1y20A', '1y7mA', '1y7yA', '1y80A', '1y82A', '1y9uA', '1yc9A', '1ydxA', '1ye5A',
                        '1yf2A', '1yg2A', '1yioA', '1yiqA', '1ylfA', '1yoyA', '1yspA', '1ysqA', '1yviA', '1z05A', '1z19A',
                        '1z7uA', '1zatA', '1zi0A', '1zljA', '1zvtA', '1zvuA', '1zzcA', '2a0bA', '2a3nA', '2a5hA', '2a5lA',
                        '2a61A', '2aa4A', '2aacA', '2ad6A', '2ad7A', '2ad8A', '2aefA', '2aejA', '2afhA', '2am1A', '2anuA',
                        '2ap1A', '2ar0A', '2araA', '2arcA', '2aznA', '2b02A', '2b0pA', '2b13A', '2b3zA', '2b44A', '2basA',
                        '2bfwA', '2bknA', '2bkoA', '2bkpA', '2bm4A', '2bm5A', '2bm6A', '2bm7A', '2bnmA', '2brcA', '2byiA',
                        '2c2aA', '2c81A', '2ce0A', '2cg4A', '2ch7A', '2cviA', '2cwqA', '2cyyA', '2d1hA', '2d1vA', '2d5mA',
                        '2d5nA', '2d5wA', '2dbbA', '2dekA', '2df8A', '2dg6A', '2di3A', '2dqlA', '2dvzA', '2dxwA', '2dxxA',
                        '2e15A', '2e1nA', '2e4nA', '2e5fA', '2e7wA', '2e7xA', '2e7zA', '2eb7A', '2ecuA', '2efnA', '2eh3A',
                        '2ehlA', '2ehzA', '2ek5A', '2eshA', '2esnA', '2esrA', '2ewnA', '2ewvA', '2eyuA', '2f00A', '2f2eA',
                        '2f5xA', '2f6gA', '2f6pA', '2f7aA', '2f7bA', '2f8lA', '2f9fA', '2fa1A', '2fa5A', '2fb2A', '2fbhA',
                        '2fcjA', '2fdnA', '2fe1A', '2fezA', '2ff4A', '2ffuA', '2fhpA', '2fn9A', '2fnuA', '2fpoA', '2fswA',
                        '2fvyA', '2fw0A', '2g2cA', '2g6vA', '2g7uA', '2gaiA', '2gajA', '2gciA', '2gd0A', '2gd2A', '2gd9A',
                        '2gj3A', '2gjgA', '2gkgA', '2glkA', '2gm5A', '2gmsA', '2gmyA', '2gqpA', '2gskA', '2gu1A', '2gufA',
                        '2guhA', '2gupA', '2gxgA', '2gzaA', '2h1cA', '2h98A', '2h99A', '2h9bA', '2hawA', '2hekA', '2heuA',
                        '2hklA', '2hmtA', '2hmuA', '2hmvA', '2hnhA', '2hoeA', '2hofA', '2hphA', '2hq0A', '2hqsA', '2hs5A',
                        '2hsgA', '2hsiA', '2hwvA', '2hxvA', '2i0mA', '2i5rA', '2ia2A', '2ia4A', '2ibdA', '2ictA', '2iftA',
                        '2ikkA', '2iplA', '2ipmA', '2ipnA', '2is1A', '2is2A', '2is4A', '2is6A', '2is8A', '2iu5A', '2iuyA',
                        '2iv7A', '2iw1A', '2iw4A', '2iwxA', '2jbaA', '2jcgA', '2jfgA', '2nipA', '2npnA', '2nq2A', '2nq9A',
                        '2nqhA', '2nt3A', '2nt4A', '2o08A', '2o0yA', '2o3jA', '2o4dA', '2o7iA', '2o7pA', '2o8xA', '2o99A',
                        '2o9aA', '2obcA', '2ofyA', '2ogiA', '2ojhA', '2okcA', '2olbA', '2oocA', '2oqgA', '2oqrA', '2oxoA',
                        '2oyoA', '2p19A', '2p4gA', '2p5vA', '2p7oA', '2paqA', '2pbqA', '2pfxA', '2ph1A', '2pjrA', '2pkhA',
                        '2pmhA', '2pn6A', '2pq7A', '2pt7A', '2pucA', '2pudA', '2px7A', '2q0oA', '2q0tA', '2q1zA', '2q4fA',
                        '2q8pA', '2qb6A', '2qb7A', '2qb8A', '2qczA', '2qdfA', '2qdlA', '2qeuA', '2qgqA', '2qgzA', '2qi9A',
                        '2qj7A', '2qm1A', '2qmoA', '2qpqA', '2qsxA', '2qwxA', '2qx4A', '2qx6A', '2qx8A', '2r01A', '2r0xA',
                        '2r1jA', '2r25A', '2r4tA', '2r6gA', '2r6oA', '2r6vA', '2ra5A', '2rb9A', '2rc7A', '2rc8A', '2rcaA',
                        '2rdeA', '2riiA', '2rilA', '2rslA', '2uagA', '2v25A', '2v2kA', '2v9yA', '2vhaA', '2vjqA', '2vk2A',
                        '2vkeA', '2vkrA', '2vlgA', '2vmaA', '2vmbA', '2vpzA', '2vshA', '2w27A', '2w8bA', '2w8iA', '2yveA',
                        '2yx0A', '2yxbA', '2yxoA', '2yxzA', '2yyeA', '2yz5A', '2z1eA', '2z1fA', '2z1uA', '2z2lA', '2z2mA',
                        '2z4gA', '2z4pA', '2z6rA', '2z8xA', '2z98A', '2z9bA', '2zauA', '2zbcA', '2zc3A', '2zc4A', '2zcmA',
                        '2zdpA', '2zf8A', '2zieA', '2zifA', '2zigA', '2zkiA', '2zkzA', '2zodA', '2zovA', '2zxjA', '3b4yA',
                        '3b6iA', '3b8xA', '3b9oA', '3bcvA', '3be6A', '3bemA', '3bg2A', '3bhqA', '3bkhA', '3bkvA', '3bm7A',
                        '3bpkA', '3bpqA', '3bpvA', '3bqxA', '3breA', '3bs3A', '3bvpA', '3bwgA', '3c1qA', '3c29A', '3c3wA',
                        '3c48A', '3c57A', '3c7jA', '3c85A', '3c8fA', '3c8nA', '3c9uA', '3canA', '3ccgA', '3cijA', '3cixA',
                        '3ckjA', '3cknA', '3ckvA', '3cloA', '3cnrA', '3cnvA', '3cp5A', '3ctpA', '3cuoA', '3cwrA', '3cx4A',
                        '3cyiA', '3cypA', '3cyqA', '3d5kA', '3d6zA', '3d7iA', '3dboA', '3df7A', '3df8A', '3dmaA', '3dr4A',
                        '3drfA', '3drjA', '3dsgA', '3du1A', '3e10A', '3e38A', '3e4rA', '3e4vA', '3e7lA', '3e8oA', '3eagA',
                        '3ec2A', '3eccA', '3echA', '3ecpA', '3edpA', '3eetA', '3efmA', '3eiwA', '3eixA', '3ekoA', '3elkA',
                        '3eusA', '3ex8A', '3eywA', '3ezuA', '3f1cA', '3f1nA', '3f1oA', '3f1pA', '3f2bA', '3f44A', '3f52A',
                        '3f6cA', '3f6oA', '3f6vA', '3f8bA', '3f8cA', '3f8fA', '3fd3A', '3fgvA', '3fisA', '3fmsA', '3fwyA',
                        '3fwzA', '3fxaA', '3fzvA', '3g13A', '3g5oA', '3g7rA', '3gdiA', '3gfaA', '3gfvA', '3gfxA', '3gfyA',
                        '3gfzA', '3gg0A', '3gg1A', '3gg2A', '3ghjA', '3gp4A', '3gpvA', '3gr3A', '3guvA', '3h4oA', '3h5tA',
                        '3h87A', '3hfiA', '3hh0A', '3hhhA', '3hl0A', '3hmzA', '3hn7A', '3hoiA', '3htvA', '3hvwA', '3pypA',
                        '3uagA', '4aahA', '4crxA', '4reqA', '4uagA', '5reqA', '6reqA', '7reqA', '8abpA']
if not os.path.isfile(large_list_fn):
    with open(large_list_fn, 'wb') as large_list_handle:
        for p_id in proteins_of_interest:
            large_list_handle.write('{}\n'.format(p_id))

In [3]:
from time import time
from DataSetGenerator import DataSetGenerator
start = time()
generator = DataSetGenerator(input_dir)
df = generator.build_pdb_alignment_dataset(protein_list_fn=os.path.basename(large_list_fn), num_threads=10,
                                           database='customuniref90.fasta', max_target_seqs=2500, remote=False,
                                           verbose=False)
end = time()
print('It took {} min to generate the data set.'.format((end - start) / 60.0))
df.to_csv('/home/daniel/Desktop/large_data_set_summary.tsv', sep='\t', index=False, header=True,
          columns=['Protein_ID', 'BLAST_Hits', 'Filtered_BLAST', 'Filtered_Alignment'])

Importing protein list
Desired structure doesn't exists
Desired structure doesn't exists
BLASTing query sequences
Filtering BLAST hits, aligning, filtering by identity, and re-aligning
It took 5.64756018321 min to generate the data set.


In [4]:
output_dir = os.environ.get('OUTPUT_PATH')
large_set_out_dir = os.path.join(output_dir, 'LargeTestSet')
if not os.path.isdir(large_set_out_dir):
    os.makedirs(large_set_out_dir)
from SeqAlignment import SeqAlignment

# Generating Values For Comparision#
To determine the effectiveness of the new method and implementation the covariation of the same proteins will be computed using the previous Evolutionary Trace covariation method (ET-MIp) and other methods in the field.

## ET-MIp##
Scoring the the covariation of the proteins using the previous Evolutionary Trace covariation method (ET-MIp).

In [6]:
from ETMIPWrapper import ETMIPWrapper
etmip_out_dir = os.path.join(large_set_out_dir, 'ET-MIp')
if not os.path.isdir(etmip_out_dir):
    os.makedirs(etmip_out_dir)
etmip_scores = {}
counts = {'success':0, 'value': 0, 'attribute':0}
for p_id in generator.protein_data:
    print('Attempting to calculate ET-MIp covariance for: {}'.format(p_id))
    try:
        protein_out_dir = os.path.join(etmip_out_dir, p_id)
        if not os.path.isdir(protein_out_dir):
            os.makedirs(protein_out_dir)
        curr_aln = SeqAlignment(file_name=generator.protein_data[p_id]['Final_FA_Aln'], query_id=p_id, polymer_type='Protein')
        curr_aln.import_alignment()
        curr_etmip = ETMIPWrapper(alignment=curr_aln)
        curr_etmip.calculate_scores(out_dir=protein_out_dir, delete_files=False)
        etmip_scores[p_id] = curr_etmip
        print('Successfully computed ET-MIp covariance for: {}'.format(p_id))
        counts['success'] += 1
    except ValueError:
        print('Could not compute ET-MIp covariance for: {} with seq_length: {} and size: {}'.format(
            p_id, curr_aln.seq_length, curr_aln.size))
        counts['value'] += 1
    except AttributeError:
        print('Could not compute ET-MIp covariance for: {} no alignment'.format(p_id))
        counts['attribute'] += 1
print('{}\tSuccesses\n{}\tValue Errors\n{}\tAttribute Errors'.format(counts['success'], counts['value'],
                                                                     counts['attribute']))

Attempting to calculate ET-MIp covariance for: 1gbs
1111.73563814
Successfully computed ET-MIp covariance for: 1gbs
Attempting to calculate ET-MIp covariance for: 2bko
578.841569185
Successfully computed ET-MIp covariance for: 2bko
Attempting to calculate ET-MIp covariance for: 2bkn
526.34930706
Successfully computed ET-MIp covariance for: 2bkn
Attempting to calculate ET-MIp covariance for: 2bkp
594.30842495
Successfully computed ET-MIp covariance for: 2bkp
Attempting to calculate ET-MIp covariance for: 2e1n
60.7038440704
Successfully computed ET-MIp covariance for: 2e1n
Attempting to calculate ET-MIp covariance for: 2yz5
781.388123035
Successfully computed ET-MIp covariance for: 2yz5
Attempting to calculate ET-MIp covariance for: 1p3d


KeyboardInterrupt: 

## DCA##
Scoring the the covariation of the proteins using a DCA julia implementation.

In [7]:
from DCAWrapper import DCAWrapper
dca_out_dir = os.path.join(large_set_out_dir, 'DCA')
if not os.path.isdir(dca_out_dir):
    os.makedirs(dca_out_dir)
dca_scores = {}
counts = {'success':0, 'value': 0, 'attribute':0}
for p_id in generator.protein_data:
    print('Attempting to calculate DCA covariance for: {}'.format(p_id))
    try:
        protein_out_dir = os.path.join(dca_out_dir, p_id)
        if not os.path.isdir(protein_out_dir):
            os.makedirs(protein_out_dir)
        curr_aln = SeqAlignment(file_name=generator.protein_data[p_id]['Final_FA_Aln'], query_id=p_id,
                                polymer_type='Protein')
        curr_aln.import_alignment()
        # Since the DCA implementation used here does not provide a way to specify the query sequence we remove the gaps
        # from the query sequences so positions will be referenced correctly for that sequence (and unnecessary
        # computations can be avoided).
        curr_aln = curr_aln.remove_gaps()
        new_aln_fn = os.path.join(protein_out_dir, '{}_no_gap.fasta'.format(p_id))
        curr_aln.write_out_alignment(new_aln_fn)
        curr_aln.file_name = new_aln_fn
        curr_dca = DCAWrapper(alignment=curr_aln)
        curr_dca.calculate_scores(out_dir=protein_out_dir, delete_file=False)
        dca_scores[p_id] = curr_dca
        print('Successfully computed DCA covariance for: {}'.format(p_id))
        counts['success'] += 1
    except ValueError:
        print('Could not compute DCA covariance for: {} with seq_length: {} and size: {}'.format(
            p_id, curr_aln.seq_length, curr_aln.size))
        counts['value'] += 1
    except AttributeError:
        print('Could not compute DCA covariance for: {} no alignment'.format(p_id))
        counts['attribute'] += 1
print('{}\tSuccesses\n{}\tValue Errors\n{}\tAttribute Errors'.format(counts['success'], counts['value'],
                                                                     counts['attribute']))

Attempting to calculate DCA covariance for: 1gbs
Removing gaps took 0.00528808434804 min
6.13056397438
Successfully computed DCA covariance for: 1gbs
Attempting to calculate DCA covariance for: 2bko
Removing gaps took 0.00132713317871 min
5.98800396919
Successfully computed DCA covariance for: 2bko
Attempting to calculate DCA covariance for: 2bkn
Removing gaps took 0.00125818252563 min
6.75035691261
Successfully computed DCA covariance for: 2bkn
Attempting to calculate DCA covariance for: 2bkp
Removing gaps took 0.00136568546295 min
6.07216501236
Successfully computed DCA covariance for: 2bkp
Attempting to calculate DCA covariance for: 2e1n
Removing gaps took 0.00021744966507 min
4.86436295509
Successfully computed DCA covariance for: 2e1n
Attempting to calculate DCA covariance for: 2yz5
Removing gaps took 0.00188196897507 min
7.43480801582
Successfully computed DCA covariance for: 2yz5
Attempting to calculate DCA covariance for: 1p3d
Removing gaps took 0.107965234915 min
15.7222940922

Removing gaps took 0.0052730679512 min
5.62178015709
Successfully computed DCA covariance for: 2z2l
Attempting to calculate DCA covariance for: 1gut
Removing gaps took 0.0103954831759 min
4.86350798607
Successfully computed DCA covariance for: 1gut
Attempting to calculate DCA covariance for: 1gus
Removing gaps took 0.01034176747 min
4.89446306229
Successfully computed DCA covariance for: 1gus
Attempting to calculate DCA covariance for: 1q35
Removing gaps took 0.0601712505023 min
9.52433896065
Successfully computed DCA covariance for: 1q35
Attempting to calculate DCA covariance for: 1gun
Removing gaps took 0.0133491317431 min
4.87783479691
Successfully computed DCA covariance for: 1gun
Attempting to calculate DCA covariance for: 2iv7
Removing gaps took 0.0109181165695 min
10.6493349075
Successfully computed DCA covariance for: 2iv7
Attempting to calculate DCA covariance for: 2qsx
Removing gaps took 0.000110332171122 min
5.81025886536
Successfully computed DCA covariance for: 2qsx
Attemp

Removing gaps took 0.00651651620865 min
4.63943886757
Successfully computed DCA covariance for: 2o8x
Attempting to calculate DCA covariance for: 2ap1
Removing gaps took 0.0198102672895 min
8.04006910324
Successfully computed DCA covariance for: 2ap1
Attempting to calculate DCA covariance for: 2fpo
Removing gaps took 0.0582794825236 min
5.83654499054
Successfully computed DCA covariance for: 2fpo
Attempting to calculate DCA covariance for: 2cg4
Removing gaps took 0.0015621026357 min
5.25553178787
Successfully computed DCA covariance for: 2cg4
Attempting to calculate DCA covariance for: 1sqe
Removing gaps took 0.000138016541799 min
4.80120491982
Successfully computed DCA covariance for: 1sqe
Attempting to calculate DCA covariance for: 1vct
Removing gaps took 0.00129753351212 min
5.67184710503
Successfully computed DCA covariance for: 1vct
Attempting to calculate DCA covariance for: 3ckj
Removing gaps took 0.0267079512278 min
7.7548289299
Successfully computed DCA covariance for: 3ckj
Att

Removing gaps took 0.00885908206304 min
5.69981384277
Successfully computed DCA covariance for: 3hmz
Attempting to calculate DCA covariance for: 2qdl
Removing gaps took 0.0131465355555 min
5.40492892265
Successfully computed DCA covariance for: 2qdl
Attempting to calculate DCA covariance for: 1xd7
Removing gaps took 0.00466785033544 min
4.83275413513
Successfully computed DCA covariance for: 1xd7
Attempting to calculate DCA covariance for: 1pb8
Removing gaps took 0.000110483169556 min
7.1048579216
Successfully computed DCA covariance for: 1pb8
Attempting to calculate DCA covariance for: 2ecu
Removing gaps took 0.00274661779404 min
5.26721000671
Successfully computed DCA covariance for: 2ecu
Attempting to calculate DCA covariance for: 3e7l
Removing gaps took 0.00151958465576 min
4.56556296349
Successfully computed DCA covariance for: 3e7l
Attempting to calculate DCA covariance for: 2aef
Removing gaps took 2.62022018433e-05 min
6.20022892952
Successfully computed DCA covariance for: 2aef

Removing gaps took 0.00435673395793 min
5.13427495956
Successfully computed DCA covariance for: 3f1o
Attempting to calculate DCA covariance for: 2gu1
Removing gaps took 0.00893706878026 min
9.65498280525
Successfully computed DCA covariance for: 2gu1
Attempting to calculate DCA covariance for: 2f00
Removing gaps took 0.122641034921 min
16.5408918858
Successfully computed DCA covariance for: 2f00
Attempting to calculate DCA covariance for: 1g6o
Removing gaps took 0.000219480196635 min
9.30754494667
Successfully computed DCA covariance for: 1g6o
Attempting to calculate DCA covariance for: 1sum
Removing gaps took 1.47183736165e-05 min
6.35862517357
Successfully computed DCA covariance for: 1sum
Attempting to calculate DCA covariance for: 1di7
Removing gaps took 0.0436289985975 min
6.46029496193
Successfully computed DCA covariance for: 1di7
Attempting to calculate DCA covariance for: 1m68
Removing gaps took 0.00226889848709 min
7.0357708931
Successfully computed DCA covariance for: 1m68
A

...

Removing gaps took 0.0306481679281 min
5.40996098518
Successfully computed DCA covariance for: 2v2k
Attempting to calculate DCA covariance for: 2zxj
Removing gaps took 0.0316046317418 min
5.31887793541
Successfully computed DCA covariance for: 2zxj
Attempting to calculate DCA covariance for: 2zif
Removing gaps took 0.00109990040461 min
7.19259810448
Successfully computed DCA covariance for: 2zif
Attempting to calculate DCA covariance for: 1kb0
Removing gaps took 0.0800989985466 min
33.7538030148
Successfully computed DCA covariance for: 1kb0
Attempting to calculate DCA covariance for: 2heu
Removing gaps took 0.000415635108948 min
11.7461061478
Successfully computed DCA covariance for: 2heu
Attempting to calculate DCA covariance for: 2ehz
Removing gaps took 0.0005717198054 min
9.40959191322
Successfully computed DCA covariance for: 2ehz
Attempting to calculate DCA covariance for: 2vkr
Removing gaps took 0.00121239821116 min
5.00867295265
Successfully computed DCA covariance for: 2vkr
At

Removing gaps took 0.0737555027008 min
6.79875898361
Successfully computed DCA covariance for: 2gqp
Attempting to calculate DCA covariance for: 1b9n
Removing gaps took 0.00237729946772 min
7.25578904152
Successfully computed DCA covariance for: 1b9n
Attempting to calculate DCA covariance for: 1d4a
Removing gaps took 0.0115499655406 min
8.17898917198
Successfully computed DCA covariance for: 1d4a
Attempting to calculate DCA covariance for: 1b9m
Removing gaps took 0.00356451670329 min
7.65679192543
Successfully computed DCA covariance for: 1b9m
Attempting to calculate DCA covariance for: 1crz
Removing gaps took 0.0604465643565 min
12.9043779373
Successfully computed DCA covariance for: 1crz
Attempting to calculate DCA covariance for: 1sly
Removing gaps took 0.00143563350042 min
28.3508031368
Successfully computed DCA covariance for: 1sly
Attempting to calculate DCA covariance for: 1crx
Removing gaps took 0.000311048825582 min
9.6809990406
Successfully computed DCA covariance for: 1crx
At

Removing gaps took 0.0266339500745 min
5.44911885262
Successfully computed DCA covariance for: 2f2e
Attempting to calculate DCA covariance for: 2c2a
Removing gaps took 0.00268241961797 min
6.89598894119
Successfully computed DCA covariance for: 2c2a
Attempting to calculate DCA covariance for: 1zvu
Removing gaps took 0.118070165316 min
34.9868938923
Successfully computed DCA covariance for: 1zvu
Attempting to calculate DCA covariance for: 1zvt
Removing gaps took 0.0189162015915 min
7.30407595634
Successfully computed DCA covariance for: 1zvt
Attempting to calculate DCA covariance for: 3ex8
Removing gaps took 0.0861447175344 min
11.1595909595
Successfully computed DCA covariance for: 3ex8
Attempting to calculate DCA covariance for: 1lr0
Removing gaps took 0.00137184858322 min
5.12512207031
Successfully computed DCA covariance for: 1lr0
Attempting to calculate DCA covariance for: 3bqx
Removing gaps took 0.0292774677277 min
5.29671311378
Successfully computed DCA covariance for: 3bqx
Attem

Removing gaps took 0.0585659662882 min
7.39571094513
Successfully computed DCA covariance for: 2eyu
Attempting to calculate DCA covariance for: 2o9a
Removing gaps took 0.0062559167544 min
6.01629900932
Successfully computed DCA covariance for: 2o9a
Attempting to calculate DCA covariance for: 1ylf
Removing gaps took 0.00652298529943 min
5.14624094963
Successfully computed DCA covariance for: 1ylf
Attempting to calculate DCA covariance for: 2is8
Removing gaps took 0.0508309205373 min
5.89537215233
Successfully computed DCA covariance for: 2is8
Attempting to calculate DCA covariance for: 2is6
Removing gaps took 0.12817500035 min
32.0242378712
Successfully computed DCA covariance for: 2is6
Attempting to calculate DCA covariance for: 2ce0
Removing gaps took 0.000780749320984 min
4.84812903404
Successfully computed DCA covariance for: 2ce0
Attempting to calculate DCA covariance for: 2is2
Removing gaps took 0.147209084034 min
31.5104899406
Successfully computed DCA covariance for: 2is2
Attemp

Removing gaps took 0.00614148378372 min
8.79939293861
Successfully computed DCA covariance for: 2ipn
Attempting to calculate DCA covariance for: 2f6g
Removing gaps took 0.00183896621068 min
6.60032010078
Successfully computed DCA covariance for: 2f6g
Attempting to calculate DCA covariance for: 1q7e
Removing gaps took 0.0104800462723 min
12.7960970402
Successfully computed DCA covariance for: 1q7e
Attempting to calculate DCA covariance for: 2yxb
Removing gaps took 0.0199309666951 min
5.52853083611
Successfully computed DCA covariance for: 2yxb
Attempting to calculate DCA covariance for: 3hoi
Removing gaps took 0.00717261632284 min
6.09053301811
Successfully computed DCA covariance for: 3hoi
Attempting to calculate DCA covariance for: 2fhp
Removing gaps took 0.0490924159686 min
6.19528603554
Successfully computed DCA covariance for: 2fhp
Attempting to calculate DCA covariance for: 2r6o
Removing gaps took 0.038244831562 min
7.79850387573
Successfully computed DCA covariance for: 2r6o
Atte

## EVCouplings##
Scoring the the covariation of the proteins using the EVCouplings method.